# Introduction


This notebook shows how to run a strategy against a live feed coming from the Binance crypto exchange. Roboquant includes a dedicated module for crypto trading, including the support for Binance that we'll use in this notebook

In [ ]:
%use roboquant
Welcome()

In [ ]:
import org.roboquant.binance.*

# Setup
We now create, as usual, the components we want to use. We start with defining a simple strategy that uses moving average. Because we won't be using USD as a base currency, we create a new Simbroker instance with an initial deposit of 1_000.000,- Binance USD (BUSD).

In [ ]:
val strategy = EMACrossover(3, 5)
val broker = SimBroker.withDeposit(1_000_000.00, "BUSD")
val roboquant =  Roboquant(strategy, ProgressMetric(), broker = broker)

# Binance Historic Feed
We start with retrieving historic data from Binance. Please note how to define a currency-pair: CURRENCY1-CURRENCY2

In [ ]:
val feed = BinanceHistoricFeed()
val tf = TimeFrame.pastPeriod(500)
feed.retrieve("BTC-BUSD", timeFrame = tf)
feed.timeFrame

In [ ]:
PriceBarChart(feed, feed.assets.first())

In [ ]:
roboquant.run(feed)

In [ ]:
roboquant.logger.summary()

In [ ]:
PriceChart(feed, feed.assets.first(), broker.account.trades)

# Binance Live Feed
Now we create an instance of the BinanceFeed and subscribe to pricebar events (candlesticks) of the the BTC/BUSD currency pair. It is also possible to subscribe to multiple currency pairs by adding additional parameters to the same *subscribePriceBar* method.

In [ ]:
val feed = BinanceFeed()
feed.subscribePriceBar("BTC-BUSD")

Because we won't be using USD as a base currency, we create a new Simbroker instance with an initial deposit of 100.000,- Binance USD (BUSD). We also define some metrics that will help afterwards to check some statistics:

1) ProgressMetric: overall progress of the run
2) PriceMetric: capture the recorded prices for an asset

In [ ]:
val strategy = EMACrossover(3, 5)
val broker = SimBroker.withDeposit(1_000_000.00, "BUSD")
val priceMetric = PriceMetric(feed.assets.first())
val roboquant =  Roboquant(strategy, ProgressMetric(), priceMetric, broker = broker)

All that remains is to start the run. Because it is a live feed we have to specify for how long we want to run it otherwise it would run forever or you interrupt the execution. In this case we run it for 30 minutes, but you can run it of course for any duration. If you run it for a short time, it is less likely that the used strategy would generate signals.

In [ ]:
val timeFrame = TimeFrame.nextMinutes(30)
roboquant.run(feed, timeFrame)

In [ ]:
roboquant.logger.summary()

In [ ]:
PriceBarChart(priceMetric, priceMetric.asset)

In [ ]:
PriceChart(priceMetric, priceMetric.asset, roboquant.broker.account.trades)

# Binance Trading
This is still work in progress and is currently not yet available. However the goal is that in the end you can also place orders on Binance using roboquant. There is already some ground work done, so have a look at the roboquant source code if you want to see what is already there and perhaps contribute some code. 